In [19]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import psycopg2

In [20]:
#read the csv
hotel_df = pd.read_csv('hotel_data.csv')
hotel_df.head()

,address,categories,city,country,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateAdded,reviews.doRecommend,reviews.id,reviews.rating,reviews.text,reviews.title,reviews.userCity,reviews.username,reviews.userProvince
0,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-09-22T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,4.0,Pleasant 10 min walk along the sea front to th...,Good location away from the crouds,NaN,Russ (kent),NaN
1,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-04-03T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Really lovely hotel. Stayed on the very top fl...,Great hotel with Jacuzzi bath!,NaN,A Traveler,NaN
2,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2014-05-13T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,Ett mycket bra hotell. Det som drog ner betyge...,Lugnt l��ge,NaN,Maud,NaN
3,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2013-10-27T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,Good location on the Lido.,NaN,Julie,NaN
4,Riviera San Nicol 11/a,Hotels,Mableton,US,45.421611,12.376187,Hotel Russo Palace,30126,GA,2015-03-05T00:00:00Z,2016-10-24T00:00:25Z,NaN,NaN,5.0,We stayed here for four nights in October. The...,������ ���������������,NaN,sungchul,NaN


In [22]:
reduced_hotel_df = hotel_df.loc[:, ["city","name","reviews.date","reviews.rating"]]
reduced_hotel_df.head()

,city,name,reviews.date,reviews.rating
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0


In [23]:
replace_zero_df = reduced_hotel_df.replace(0, np.nan)
hotel_drop_rating_na_df = replace_zero_df.dropna(subset = ['reviews.rating'])
hotel_drop_rating_na_df.head()

,city,name,reviews.date,reviews.rating
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0


In [24]:
average_rating_df = hotel_drop_rating_na_df.groupby('name').aggregate({'reviews.rating': 'mean'}).reset_index()
average_rating_df = average_rating_df.rename(columns={'reviews.rating':'average_rating'})
average_rating_df

,name,average_rating
0,1785 Inn,3.500000
1,1900 House,4.923077
2,40 Berkeley Hostel,3.329193
3,A Bed & Breakfast In Cambridge,3.574074
4,Acorn Motor Inn,3.750000
...,...,...
626,Wingate By Wyndham Pueblo,4.433333
627,Wisconsin-aire Motel,4.285714
628,Wyndham Resort At Fairfield Mountains,3.928571
629,Wyndham Vacation Resorts Towers On The Grove,4.050000


In [25]:
agg_df = hotel_drop_rating_na_df.merge(average_rating_df)

In [26]:
agg_df = agg_df.rename(columns={'name':'hotel_name', 'reviews.date':"date_of_review", 'reviews.rating':'hotel_rating', 'average_rating':'average_rating_by_hotel'})
agg_df.head()

,city,hotel_name,date_of_review,hotel_rating,average_rating_by_hotel
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0,4.052632
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0,4.052632
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0,4.052632
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0,4.052632
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0,4.052632


In [27]:
rating_by_city_df = hotel_drop_rating_na_df.groupby('city').aggregate({'reviews.rating': 'mean'}).reset_index()
rating_by_city_df = rating_by_city_df.rename(columns={'reviews.rating':'average_rating_by_city'})
rating_by_city_df

,city,average_rating_by_city
0,Abbeville,3.637795
1,Abilene,4.147541
2,Absecon,2.454545
3,Ada,3.833333
4,Aiken,2.142857
...,...,...
530,Woodland,4.000000
531,Woodstock,4.400000
532,Worthington,2.232143
533,Yakima,1.000000


In [28]:
agg_df = agg_df.merge(rating_by_city_df)
agg_df.head()

,city,hotel_name,date_of_review,hotel_rating,average_rating_by_hotel,average_rating_by_city
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0,4.052632,4.052632
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0,4.052632,4.052632
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0,4.052632,4.052632
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0,4.052632,4.052632
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0,4.052632,4.052632


In [29]:
lambda_func = lambda x, y : x if x > y else y
higher_rating_list = []
for index,row in agg_df.iterrows():
    higher_rating_list.append(lambda_func(row['average_rating_by_hotel'], row['average_rating_by_city']))
agg_df['higher_average_rating'] = higher_rating_list

agg_df.head()

,city,hotel_name,date_of_review,hotel_rating,average_rating_by_hotel,average_rating_by_city,higher_average_rating
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0,4.052632,4.052632,4.052632
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0,4.052632,4.052632,4.052632
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0,4.052632,4.052632,4.052632
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0,4.052632,4.052632,4.052632
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0,4.052632,4.052632,4.052632


In [30]:
subtraction_func = lambda x, y : x - y
difference_list = []
for index,row in agg_df.iterrows():
    difference_val = subtraction_func(row['average_rating_by_hotel'], row['average_rating_by_city'])
    difference_list.append(abs(difference_val))
agg_df['diff_between_avg_ratings'] = difference_list

agg_df.head()

,city,hotel_name,date_of_review,hotel_rating,average_rating_by_hotel,average_rating_by_city,higher_average_rating,diff_between_avg_ratings
0,Mableton,Hotel Russo Palace,2013-09-22T00:00:00Z,4.0,4.052632,4.052632,4.052632,0.0
1,Mableton,Hotel Russo Palace,2015-04-03T00:00:00Z,5.0,4.052632,4.052632,4.052632,0.0
2,Mableton,Hotel Russo Palace,2014-05-13T00:00:00Z,5.0,4.052632,4.052632,4.052632,0.0
3,Mableton,Hotel Russo Palace,2013-10-27T00:00:00Z,5.0,4.052632,4.052632,4.052632,0.0
4,Mableton,Hotel Russo Palace,2015-03-05T00:00:00Z,5.0,4.052632,4.052632,4.052632,0.0


In [ ]:
#connect to local database
rds_connection_string = "postgres:postgres@localhost:5432/hotels_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
#check table names
engine.table_names()

In [ ]:
#load dataframe into database
agg_df.to_sql(name='csv', con=engine, if_exists='append', index=False)

In [ ]:
#confirm that dataframe has been properly loaded by querying table
pd.read_sql_query('select * from csv', con=engine).head()